In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser
from langchain_teddynote.tools import GoogleNews
from langchain.agents import tool
from typing import List, Dict
from bs4 import BeautifulSoup
import requests
import re

In [3]:
@tool
def get_word_length(word: str) -> int: 
    """Returns the length of a word."""
    return len(word)

In [4]:
@tool
def add_function(a: float, b: float) -> float:
    """Adds two numbers together."""
    return a + b

In [5]:
@tool
def naver_news_crawl(news_url: str) -> str:
    """Crawls a 네이버 (naver.com) news article and returns the body content."""
    
    response = requests.get(news_url)                           # HTTP GET 요청 보내기

    if response.status_code == 200:                             # 요청 성공        
        soup = BeautifulSoup(response.text, "html.parser")      # BeautifulSoup을 사용하여 HTML 파싱

        # 원하는 정보 추출
        title = soup.find("h2", id="title_area").get_text()
        content = soup.find("div", id="contents").get_text()
        cleaned_title = re.sub(r"\n{2,}", "\n", title)
        cleaned_content = re.sub(r"\n{2,}", "\n", content)
    else:
        print(f"HTTP 요청 실패. 응답 코드: {response.status_code}")

    return f"{cleaned_title}\n{cleaned_content}"

In [6]:
@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    # 키워드로 뉴스 검색하는 도구 생성     
    """Look up news by keyword"""
    print(query)
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)

In [7]:
tools = [get_word_length, add_function, naver_news_crawl, search_keyword]

In [8]:
def execute_tool_calls(tool_call_results):
    """
    도구 호출 결과를 실행하는 함수

    :param tool_call_results: 도구 호출 결과 리스트
    :param tools: 사용 가능한 도구 리스트
    """    
    for tool_call_result in tool_call_results:          # 도구 호출 결과 리스트를 순회합니다.
        # 도구의 이름과 인자를 추출합니다.
        tool_name = tool_call_result["type"]            # 도구의 이름(함수명)
        tool_args = tool_call_result["args"]            # 도구에 전달되는 인자
        
        # 도구 이름과 일치하는 도구를 찾아 실행합니다.
        matching_tool = None

        for tool in tools:
            if tool.name == tool_name:
                matching_tool = tool
                break


        
        if matching_tool:            
            result = matching_tool.invoke(tool_args)                                        # 일치하는 도구를 찾았다면 해당 도구를 실행합니다.            
            print(f"[실행도구] {tool_name} [Argument] {tool_args}\n[실행결과] {result}")    # 실행 결과를 출력합니다.
        else:            
            print(f"경고: {tool_name}에 해당하는 도구를 찾을 수 없습니다.")                 # 일치하는 도구를 찾지 못했다면 경고 메시지를 출력합니다.
        

In [9]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)    # 모델 생성

In [10]:
llm_with_tools = llm.bind_tools(tools)                  # 도구 바인딩

In [11]:
chain = llm_with_tools | JsonOutputToolsParser(tools=tools) | execute_tool_calls

In [12]:
chain.invoke("What is the length of the word LEEINHWAN?")

[실행도구] get_word_length [Argument] {'word': 'LEEINHWAN'}
[실행결과] 9


In [13]:
chain.invoke("뉴스 기사 내용을 크롤링해줘: https://n.news.naver.com/mnews/hotissue/article/092/0002347672?type=series&cid=2000065")

[실행도구] naver_news_crawl [Argument] {'news_url': 'https://n.news.naver.com/mnews/hotissue/article/092/0002347672?type=series&cid=2000065'}
[실행결과] [미장브리핑] 9월 미국 CPI 주목…3분기 S&P500 실적 발표

			▲10일(현지시간) 미국 9월 소비자물가지수(CPI) 발표 예정. 고용 지표가 양호하게 나온 가운데 물가 지표 주목. 9월 미국 비농업고용 25만4천명 증가해 시장 예상치 14만명 크게 상회. 이는 6개월 래 최대 규모로 지난 12개월 평균값 20만3천명 증가한 것보다도 높은 수치. 9월 실업률은 4.1%로 2개월 연속 하락했으며, 평균 시간당 임금은 전년 동월 대비 4% 증가해 5월 이후 최고 수준.▲시장에서 9월 헤드라인 CPI는 8월 전년 동월 대비 2.6% 로 5개월 연속 둔화하고 9월에는 2.3% 증가로 추가 하락 예상. 전월 대비도 8월 0.2% 둔화 예상. 근원 CPI는 지난 8월 3.2%와 비슷한 수준 관측.▲11일에는 미국 9월 제조업물가지수(PPI) 발표. 지난 6월 부터 8월까지 반등 추세 꺾여. 8월은 1.7% 증가.
(사진=이미지투데이)▲11월 미국 연방준비제도(연준) 공개시장위원회(FOMC) 에서 0.50%p 인하 기대가 크케 후퇴한 가운데, 9일에는 FOMC 의사록 공개. 지난 9월 회의에서 빅컷(0.50%p) 단행한 배경과 인플레이션 전망에 대한 논의를 알 수 있을 것으로 보여.▲미국 스탠다드앤푸어스(S&P) 500 기업의 3분기 실적 발표 시작. 평균 이익증가율 추정치는 전년 동기 대비 4.6%로 5개분기 연속 플러스이나 증가폭은 둔화 예상. 11일부터 JP모건체이스, 웰스파고 등 대형은행들의 실적 발표.▲FTSE 러셀은 8일 정례 시장분류 결과를 발표. 한국은 2022년 관찰대상국 지정 이후 금번 시장접근성 등급(L1) 상향으로 세계국채지수(WGBI) 에 편입될 지 관심. 주식의 경우 지난 2009년